In [ ]:
# Import packages 
import os
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors
import matplotlib.gridspec as gsp

from scipy.io import FortranFile

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import importlib
import glob
import copy

import Plots as Po

importlib.reload(Po)

#sys.path.append('../Plotting/')
#   Now you can imprt modules in ../Plotting
#import xyp_plot as xyp
#import ana as a


In [ ]:
TopoRoot = '/project/amp/juliob/Topo-generate-devel/Topo/Topo.git/'
Case = 'fv_0.9x1.25_Sco100_SMocean'
cgrid= 'fv_0.9x1.25'
Gridkey='yx'

In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/'+cgrid+'*.nc')
FileN = foo[-1]

# OR override with actual file used by Michael
FileN = '/project/amp/juliob/bndtopo/fv_0.9x1.25_nc3000_Nsw042_Nrs008_Co060_Fi001_ZR_sgh30_24km_GRNL_MERRA2_c171218.nc'

print(FileN)
dCAM=xr.open_dataset( FileN )
print( list(dCAM.variables) )
print(np.shape(dCAM['lat']))
lonc=dCAM['lon'].values
latc=dCAM['lat'].values
angl=dCAM['ANGLL'].values
hwdt=dCAM['HWDTH'].values
clng=dCAM['CLNGT'].values
mxd=dCAM['MXDIS'].values
sgh=dCAM['SGH'].values


In [ ]:
foo=glob.glob( TopoRoot +'/cases/'+ Case + '/output/topo*.nc')
FileN = foo[-1]
print(FileN)
topo=xr.open_dataset( FileN )
print(list(topo.variables))
print(topo.attrs)
raw=topo['terr_dev']+topo['terr_sm']
Loo = np.size(raw)
nc=int(np.sqrt( Loo/6 ))
print(int(nc))
raw=np.reshape( raw.values , (6,nc,nc) )
dev=np.reshape( topo['terr_dev'].values , (6,nc,nc) )
smoo=np.reshape( topo['terr_sm'].values , (6,nc,nc) )
tlats=np.reshape( topo['lat'].values , (6,nc,nc) )
tlons=np.reshape( topo['lon'].values , (6,nc,nc) )


In [ ]:

wlon,elon=-77.,-57.
slat,nlat=-58,-45


if (elon<0.):
    elon=elon+360.
if (wlon<0.):
    wlon=wlon+360.

lonrange=np.asarray([wlon,elon] ) #-2
latrange=np.asarray([slat,nlat]) #-4

#Which panel will you be in?
ip=4

In [ ]:
if (Gridkey=='c'):
    zoo=np.where( ( (lonc<=lonrange[1])&(lonc>lonrange[0]) )&( (latc<=latrange[1])&(latc>latrange[0]) ) )
    print(len(zoo[0]))
    iroo=zoo[0]
elif (Gridkey=='yx'):
    zoox=np.where( ( (lonc<=lonrange[1])&(lonc>lonrange[0]) ) )
    zooy=np.where( ( (latc<=latrange[1])&(latc>latrange[0]) ) )
    print(len(zoox[0]),len(zooy[0]))
    iroox=zoox[0]
    irooy=zooy[0]

print(np.shape(angl))

In [ ]:
terrlev=[-100,0,10,20,50,100,200,500,1000,1500,2000,2500,3000 ] #,3500,4000]
devlev=np.linspace(0,2000,num=21)
cmap=plt.cm.terrain

"""
fig,ax = plt.subplots(2,3,figsize=(21,20) ,squeeze=False ) # , sharex=True)

"""
fig = plt.figure(figsize=(15,18) )
gs = gsp.GridSpec( 2, 2, figure=fig ,wspace=0.1, hspace=0.1  ) #, sharey=True)
    
j,i=0,0
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],raw[ip,:,:], levels=terrlev, cmap='terrain',alpha=.5) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9 , label='meters' )
ax.set_title(loc='left',label='$\\bf{A)}$ A-grid (3km cubed-sphere gnomonic grid) \n topography' , fontsize=16)
ax.set_xlabel('Longitude',fontsize=14 )
ax.tick_params(axis='x', labelsize=14)
ax.set_ylabel('Latitude',fontsize=14 )
ax.tick_params(axis='y', labelsize=14)


j,i=0,1
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],smoo[ip,:,:], levels=terrlev, cmap='terrain',alpha=.5) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9, label='meters' )
ax.set_title(loc='left',label='$\\bf{B)}$ A-grid topography smoothed with a \n radial scale $\sim$ 180km ' , fontsize=16)
ax.set_xlabel('Longitude',fontsize=14 )
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=0)

j,i=1,0
#plt.contourf( tlons[ip,:,:],tlats[ip,:,:],mxdis[ip,:,:], levels=np.linspace(-100,500.,num=21), cmap='terrain' ,alpha=.5) #,
ax=fig.add_subplot(gs[j,i])
u=ax.contourf( tlons[ip,:,:],tlats[ip,:,:],dev[ip,:,:], levels=devlev, cmap='terrain',alpha=1) #,
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9, label='meters' )
ax.set_title(loc='left',label='$\\bf{C)}$ Unresolved topography on A-grid' , fontsize=16)
ax.set_xlabel('Longitude',fontsize=14 )
ax.tick_params(axis='x', labelsize=14)
ax.set_ylabel('Latitude',fontsize=14 )
ax.tick_params(axis='y', labelsize=14)

j,i=1,1
ax=fig.add_subplot(gs[j,i])
if (Gridkey=='c'):
    u=ax.tricontourf(lonc,latc,mxd[0,:], levels=devlev, cmap='terrain',alpha=1)
    for ir in iroo:
        lat0,lon0 = latc[ir],lonc[ir]
        lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,ir], clngt=clng[0,ir] )
        plt.plot( [lon1,lon2] , [lat1,lat2], color='black' )
plt.colorbar(u,ax=ax,location='bottom',shrink=.9, label='meters' )
ax.set_title(loc='left',label='$\\bf{D)}$ Ridge height estimate (w/ ridge crests) on \n CAM/WACCM horizontal grid' , fontsize=16)
   
    
if (Gridkey=='yx'):
    u=ax.contourf(lonc,latc,mxd[0,:,:]-1, levels=devlev, cmap='terrain',alpha=1)
    for iry in irooy:
        for irx in iroox:
            lat0,lon0 = latc[iry],lonc[irx]
            lat1,lon1,lat2,lon2 = Po.CAMridgelet(lat=lat0 ,lon=lon0 , angll=angl[0,iry,irx], clngt=clng[0,iry,irx] )
            if(mxd[0,iry,irx]>100.):
                ax.plot( [lon1,lon2] , [lat1,lat2], color='black' )
    
ax.set_xlim(lonrange)
ax.set_ylim(latrange)

ax.set_xlabel('Longitude',fontsize=14 )
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=0)

"""
j,i=1,2
ax=fig.add_subplot(gs[j,i])
if (Gridkey=='yx'):
    u=ax.contourf(lonc,latc,2*sgh[:,:]-1, levels=devlev, cmap='terrain',alpha=1)
ax.set_xlim(lonrange)
ax.set_ylim(latrange)
plt.colorbar(u,ax=ax,location='bottom',shrink=.9)
ax.set_title(loc='left',label='$\\bf{E)}$ Obstacle height estimate based on grid mean standard dev on CAM/WACCM horizontal grid' , fontsize=14)
"""

#ax1 = fig.add_subplot(3, 4, ipoo , projection=ccrs.PlateCarree(central_longitude=0))
#cf = ax1.contourf(xlon, xlat, gutnx[L,:,:],  levels=clevs, cmap=cmap  , transform=ccrs.PlateCarree())